In [ ]:
# Mount Google Drive if needed (optional)
!git clone --recurse-submodules https://github.com/porwai/implicit-ins-improved.git
%cd implicit-ins-improved

fatal: destination path 'LESS' already exists and is not an empty directory.
[Errno 2] No such file or directory: 'implicit-ins-improved'
/content/implicit-ins-improved


In [ ]:
# ⚙️ STEP 0: SETUP
import os

# Move to repo root if inside notebooks/
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Install LESS and dependencies
!pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -r LESS/requirements.txt
!pip install -e LESS/
!pip install alpaca_eval datasets

# Confirm path
!ls LESS

In [ ]:
from datasets import load_dataset

openmath = load_dataset("nvidia/OpenMathInstruct-1", split="train")


In [ ]:
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

In [ ]:
from datasets import load_dataset
import json
import os

'''
# Save OpenMathInstruct-1
openmath = load_dataset("nvidia/OpenMathInstruct-1", split="train")
openmath_data = [{"messages": ex["messages"]} for ex in openmath]
with open("data/openmath_full.json", "w") as f:
    json.dump(openmath_data, f)
'''

# Save AlpacaEval as target prompts
alpaca_data = [{"input": x["instruction"], "output": ""} for x in eval_set]
os.makedirs("data/eval/alpaca_eval", exist_ok=True)
with open("data/eval/alpaca_eval/alpaca_eval.json", "w") as f:
    json.dump(alpaca_data, f)



In [ ]:
from transformers import AutoTokenizer
from less.data_selection.get_validation_dataset import get_dataset

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-rw-1b", trust_remote_code=True)

# Load validation data from AlpacaEval
val_dataset = get_dataset(
    task="alpaca_eval",
    data_dir="data",
    tokenizer=tokenizer,
    max_length=1024,
    use_chat_format=True,
    chat_format="tulu"
)


In [ ]:
!bash LESS/less/scripts/train/warmup_lora_train.sh data tiiuae/falcon-rw-1b 0.05 3 falcon-warmup

In [ ]:
CKPT=50  # replace with actual
!bash LESS/less/scripts/get_info/get_train_lora_grads.sh \
  data/openmath_full.json \
  out/falcon-warmup/checkpoint-${CKPT} \
  grads/falcon-openmath-ckpt${CKPT} \
  4096 \
  adam


In [ ]:
!bash LESS/less/scripts/get_info/get_eval_lora_grads.sh \
  alpaca_eval \
  data \
  out/falcon-warmup/checkpoint-${CKPT} \
  grads/falcon-alpaca-ckpt${CKPT}-sgd \
  "4096"


In [ ]:
!bash LESS/less/scripts/data_selection/matching.sh \
  grads/falcon-openmath-ckpt${CKPT}/dim4096 \
  openmath \
  ${CKPT} \
  1e-5 \
  grads/falcon-alpaca-ckpt${CKPT}-sgd/dim4096 \
  alpaca_eval \
  selected_data


In [ ]:
!python3 -m less.data_selection.write_selected_data \
--target_task_names alpaca_eval \
--train_file_names openmath \
--train_files data/openmath_full.json \
--output_path selected_data \
--k 1000
